## This notebook does some data re-arrangements

- Combine all events to a single pandas dataframe, and save as `all_subjects_events.pkl`. Note that the onsets here are the original onset timings, no slice timing correction applied.
- Combine all extracted signals from the STN subregions, and save as `all_subjects_stn_timeseries.pkl`
- Combine all confounds that will be used in the analyses, and save as `all_subjects_confounds.pkl`

In [1]:
import pandas as pd
import numpy as np
import os

import glob
import re

def to_psc(x):
    return x / x.mean() * 100 - 100

## Events

In [ ]:
## get events
behavior = pd.read_pickle('./derivatives/behavior.pkl')
behavior['sub'] = behavior['subject']
behavior.loc[behavior['ds']=='ds-02', 'sub'] = behavior.loc[behavior['ds']=='ds-02', 'sub'].astype(int) + behavior.loc[behavior['ds']=='ds-01', 'sub'].astype(int).max()
behavior['sub'] = behavior['sub'].astype(int)
behavior['onset_response'] = behavior['onset_stim'] + behavior['rt']/1000
behavior = behavior.loc[behavior.response.isin([1,2])]  # remove null-responses

# generate feedback column
behavior['feedback'] = 'feedback_+0'
behavior.loc[(behavior['correct'] == 1) & (behavior['cue congruency'] == 'congruent'), 'feedback'] = 'feedback_+0.04'
behavior.loc[(behavior['correct'] == 1) & (behavior['cue congruency'] == 'incongruent'), 'feedback'] = 'feedback_+0.01'
behavior.loc[(behavior['correct'] == 1) & (behavior['cue congruency'] == 'neutral'), 'feedback'] = 'feedback_+0.025'
events_cue = behavior[['sub', 'block', 'cue', 'onset_cue']]
events_cue['event_type'] = events_cue['cue'].apply(lambda x: 'cue_' + x)
events_cue = events_cue.rename(columns={'block': 'run', 'onset_cue': 'onset'})
events_cue['duration'] = 1

events_stim = behavior[['sub', 'block', 'difficulty', 'onset_stim']]
events_stim = events_stim.rename(columns={'block': 'run', 'onset_stim': 'onset', 'difficulty': 'event_type'})
events_stim['duration'] = 1.5

events_accuracy = behavior[['sub', 'block', 'correct', 'onset_stim']]
events_accuracy['event_type'] = events_accuracy['correct'].replace({1: 'correct', 0: 'error'})
events_accuracy = events_accuracy.rename(columns={'block': 'run', 'onset_stim': 'onset', 'accuracy': 'event_type'})
events_accuracy['onset'] += 1.5   # NB: "Error" is only known at the feedback onset, not at the stimulus onset
events_accuracy['duration'] = 0.5

events_fb = behavior[['sub', 'block', 'feedback', 'onset_stim']]
events_fb = events_fb.rename(columns={'block': 'run', 'onset_stim': 'onset', 'feedback': 'event_type'})
events_fb['onset'] += 1.5         # NB: feedback is only shown at the feedback onset, not at the stimulus onset
events_fb['duration'] = 0.5

events_responses = behavior[['sub', 'block', 'response', 'onset_response']]
events_responses['event_type'] = events_responses['response'].replace({1: 'response_left', 2: 'response_right'})
events_responses = events_responses.rename(columns={'block': 'run', 'onset_response': 'onset'})
events_responses['duration'] = 0.01 # model as stick function

events = pd.concat((events_cue, events_stim, events_accuracy, events_responses, events_fb))
events = events.sort_values(by=['sub', 'run', 'onset'])
events = events.rename(columns={'sub': 'subject'})

events = events.set_index(['subject', 'run'])[['onset', 'event_type', 'duration']]#.rename(columns={'event_type': 'trial_type'})
events['duration'] = .001
# add durations
# events.loc[events.event_type.isin(['cue_left', 'cue_right', 'cue_neutral']), 'duration'] = 1
# events.loc[events.event_type.isin(['easy', 'hard']), 'duration'] = 1.5
# events.loc[events.event_type.isin(['error', 'correct']), 'duration'] = 0.5
# events.loc[events.event_type.isin(['response_left', 'response_right']), 'duration'] = 0.001 # model as stick function

events.to_pickle('./derivatives/all_subjects_events.pkl')

events['onset'] -= 1.5  # STC
events.head()

## Extracted STN subregion signals

In [41]:
## load all extracted signals
all_csvs = sorted(glob.glob('./derivatives/ds-*/extracted_signal/sub-*/func/*T1w*subroi*.csv'))

reg = re.compile('.*ds-(?P<ds>\d+)/extracted_signal/sub-(?P<sub>\d+)/func/sub-.*_task-randomdotmotion_run-(?P<run>\d+)_space-T1w_desc-preproc_desc-stn(?P<hemisphere>\S+)_subroi-(?P<roi>\S).*')
reg.match(all_csvs[1]).groupdict()

{'ds': '01', 'sub': '01', 'run': '01', 'hemisphere': 'l', 'roi': 'B'}

In [ ]:
df = []

# loop over .csv-files, adding signal row-by-row
for fn in all_csvs:
    d = reg.match(fn).groupdict()
    d['signal'] = pd.read_csv(fn, index_col=0).apply(to_psc, axis=0).mean(1)
    d = pd.DataFrame(d)
    d['t'] = np.arange(0, d.shape[0]*3, 3)
    
    df.append(d)

df = pd.concat(df, axis=0)
df['ds'] = df['ds'].astype(int)
df['run'] = df['run'].astype(int)
df['sub'] = df['sub'].astype(int)

# adjust ds2 subject idx to ensure non-overlapping subject idx
max_sub_ds1 = df.loc[df.ds==1,'sub'].max()
df.loc[df.ds==2,'sub'] = df.loc[df.ds==2,'sub']+max_sub_ds1

df_wide = df.rename(columns={'sub':'subject'}).pivot_table(values=['signal'], index=['ds', 'subject', 'run', 't'], columns=['hemisphere', 'roi'])
df_wide = df_wide.reset_index(level=0, drop=True)
df_wide.columns = ['_'.join(col) for col in df_wide.columns]

display(df_wide.head())
df_wide.to_pickle('./derivatives/all_subjects_stn_timeseries.pkl')

## Extract entire STN signals

In [43]:
## load all extracted signals
all_csvs = sorted(glob.glob('./derivatives/ds-*/extracted_signal/sub-*/func/*T1w*desc-*.csv'))
all_csvs = [x for x in all_csvs if not 'subroi' in x]

reg = re.compile('.*ds-(?P<ds>\d+)/extracted_signal/sub-(?P<sub>\d+)/func/sub-.*_task-randomdotmotion_run-(?P<run>\d+)_space-T1w_desc-preproc_desc-stn(?P<hemisphere>\S).*')
reg.match(all_csvs[1]).groupdict()

{'ds': '01', 'sub': '01', 'run': '01', 'hemisphere': 'r'}

In [44]:
df = []

# loop over .csv-files, adding signal row-by-row
for fn in all_csvs:
    d = reg.match(fn).groupdict()
    d['signal'] = pd.read_csv(fn, index_col=0).apply(to_psc, axis=0).mean(1)
    d = pd.DataFrame(d)
    d['t'] = np.arange(0, d.shape[0]*3, 3)
    
    df.append(d)

df = pd.concat(df, axis=0)
df['ds'] = df['ds'].astype(int)
df['run'] = df['run'].astype(int)
df['sub'] = df['sub'].astype(int)

# adjust ds2 subject idx to ensure non-overlapping subject idx
max_sub_ds1 = df.loc[df.ds==1,'sub'].max()
df.loc[df.ds==2,'sub'] = df.loc[df.ds==2,'sub']+max_sub_ds1

df_wide = df.rename(columns={'sub':'subject'}).pivot_table(values=['signal'], index=['ds', 'subject', 'run', 't'], columns=['hemisphere'])
df_wide = df_wide.reset_index(level=0, drop=True)
df_wide.columns = ['_'.join(col) for col in df_wide.columns]

display(df_wide.head())
df_wide.to_pickle('./derivatives/all_subjects_whole_stn_timeseries.pkl')

signal_l  signal_r
subject run t                     
1       1   0  -1.754069 -1.423017
            3  -0.668470 -0.437848
            6  -0.734271 -0.290067
            9   0.733882 -0.120541
            12 -0.000864  0.868631

In [45]:
## no PSC
df = []

# loop over .csv-files, adding signal row-by-row
for fn in all_csvs:
    d = reg.match(fn).groupdict()
    d['signal'] = pd.read_csv(fn, index_col=0).mean(1)
    d = pd.DataFrame(d)
    d['t'] = np.arange(0, d.shape[0]*3, 3)
    
    df.append(d)

df = pd.concat(df, axis=0)
df['ds'] = df['ds'].astype(int)
df['run'] = df['run'].astype(int)
df['sub'] = df['sub'].astype(int)

# adjust ds2 subject idx to ensure non-overlapping subject idx
max_sub_ds1 = df.loc[df.ds==1,'sub'].max()
df.loc[df.ds==2,'sub'] = df.loc[df.ds==2,'sub']+max_sub_ds1

df_wide = df.rename(columns={'sub':'subject'}).pivot_table(values=['signal'], index=['ds', 'subject', 'run', 't'], columns=['hemisphere'])
df_wide = df_wide.reset_index(level=0, drop=True)
df_wide.columns = ['_'.join(col) for col in df_wide.columns]

display(df_wide.head())
df_wide.to_pickle('./derivatives/all_subjects_whole_stn_timeseries_nopsc.pkl')

signal_l     signal_r
subject run t                           
1       1   0   1383.694961  1511.894859
            3   1398.178735  1526.361747
            6   1398.672099  1529.155277
            9   1417.989671  1531.482607
            12  1407.499801  1546.246835

## Confounds

In [ ]:
## get confounds
all_csvs = glob.glob('./derivatives/ds-*/fmriprep/sub-*/func/sub-*_task-randomdotmotion_run-*_desc-confounds_regressors.tsv')

reg = re.compile('.*ds-(?P<ds>\d+)/fmriprep/sub-.*/func/sub-(?P<sub>\d+)_.*_run-(?P<run>\d+)_.*')
reg.match(all_csvs[1]).groupdict()

confounds_df = []

# loop over .csv-files, adding signal row-by-row
for fn in all_csvs:
    d = reg.match(fn).groupdict()
    tmp = pd.read_csv(fn, sep='\t')
    tmp['sub'] = d['sub']
    tmp['run'] = d['run']
    tmp['ds'] = d['ds']
    confounds_df.append(tmp)

confounds_df = pd.concat(confounds_df, axis=0)
confounds_df['ds'] = confounds_df['ds'].astype(int)
confounds_df['run'] = confounds_df['run'].astype(int)
confounds_df['subject'] = confounds_df['sub'].astype(int)

# adjust ds2 subject idx to ensure non-overlapping subject idx
max_sub_ds1 = confounds_df.loc[confounds_df.ds==1,'subject'].max()
confounds_df.loc[confounds_df.ds==2,'subject'] = confounds_df.loc[confounds_df.ds==2,'subject']+max_sub_ds1
confounds_df = confounds_df.sort_values(['ds', 'subject', 'run']).set_index(['ds', 'subject', 'run'])

# Include confounds: 5 ACompCor regressors, cosines for drift, DVARS, rotation & translation params
confounds = confounds_df.reset_index(level=0, drop=True)
include_confounds = ['a_comp_cor_0' + str(x) for x in range(5)] + ['cosine' + str(x).zfill(2) for x in range(17)]
include_confounds = include_confounds + ['dvars', 'framewise_displacement']
include_confounds = include_confounds + ['rot_' + a for a in ['x', 'y', 'z']]
include_confounds = include_confounds + ['trans_' + a for a in ['x', 'y', 'z']]
confounds = confounds[include_confounds]
confounds.head()

# despite being complete runs, for subjects 9 run 1 and subject 21 run 1 cosine16 were not defined. Copy from another subject
confounds.loc[(9, 1),'cosine16'] = confounds.loc[(1,1), 'cosine16'].values.copy()
confounds.loc[(21, 1),'cosine16'] = confounds.loc[(1,1), 'cosine16'].values.copy()

# fill NA of dvars with mean value per run, and 0 of framewise displacement of first volume
def fillna(x):
    x.loc[pd.isnull(x['dvars']), 'dvars'] = np.nanmean(x['dvars'])
    x['framewise_displacement'] = x['framewise_displacement'].fillna(0)
    return x

confounds = confounds.reset_index()
confounds[['dvars', 'framewise_displacement']] = confounds.groupby(['subject', 'run'])[['dvars', 'framewise_displacement']].apply(lambda x: fillna(x))
confounds = confounds.set_index(['subject', 'run'])

confounds.to_pickle('./derivatives/all_subjects_confounds.pkl')

## confounds with global signal
**Not used in paper, only for a control analysis**

In [7]:
## get confounds
all_csvs = glob.glob('./derivatives/ds-*/fmriprep/sub-*/func/sub-*_task-randomdotmotion_run-*_desc-confounds_regressors.tsv')

reg = re.compile('.*ds-(?P<ds>\d+)/fmriprep/sub-.*/func/sub-(?P<sub>\d+)_.*_run-(?P<run>\d+)_.*')
reg.match(all_csvs[1]).groupdict()

confounds_df = []

# loop over .csv-files, adding signal row-by-row
for fn in all_csvs:
    d = reg.match(fn).groupdict()
    tmp = pd.read_csv(fn, sep='\t')
    tmp['sub'] = d['sub']
    tmp['run'] = d['run']
    tmp['ds'] = d['ds']
    confounds_df.append(tmp)

confounds_df = pd.concat(confounds_df, axis=0)
confounds_df['ds'] = confounds_df['ds'].astype(int)
confounds_df['run'] = confounds_df['run'].astype(int)
confounds_df['subject'] = confounds_df['sub'].astype(int)

# adjust ds2 subject idx to ensure non-overlapping subject idx
max_sub_ds1 = confounds_df.loc[confounds_df.ds==1,'subject'].max()
confounds_df.loc[confounds_df.ds==2,'subject'] = confounds_df.loc[confounds_df.ds==2,'subject']+max_sub_ds1
confounds_df = confounds_df.sort_values(['ds', 'subject', 'run']).set_index(['ds', 'subject', 'run'])

# Include confounds: 5 ACompCor regressors, cosines for drift, DVARS, rotation & translation params
confounds = confounds_df.reset_index(level=0, drop=True)
include_confounds = ['a_comp_cor_0' + str(x) for x in range(5)] + ['cosine' + str(x).zfill(2) for x in range(17)]
include_confounds = include_confounds + ['dvars', 'framewise_displacement']
include_confounds = include_confounds + ['rot_' + a for a in ['x', 'y', 'z']]
include_confounds = include_confounds + ['trans_' + a for a in ['x', 'y', 'z']]
include_confounds += ['global_signal']
confounds = confounds[include_confounds]
confounds.head()

# despite being complete runs, for subjects 9 run 1 and subject 21 run 1 cosine16 were not defined. Copy from another subject
confounds.loc[(9, 1),'cosine16'] = confounds.loc[(1,1), 'cosine16'].values.copy()
confounds.loc[(21, 1),'cosine16'] = confounds.loc[(1,1), 'cosine16'].values.copy()

# fill NA of dvars with mean value per run, and 0 of framewise displacement of first volume
def fillna(x):
    x.loc[pd.isnull(x['dvars']), 'dvars'] = np.nanmean(x['dvars'])
    x['framewise_displacement'] = x['framewise_displacement'].fillna(0)
    return x

confounds = confounds.reset_index()
confounds[['dvars', 'framewise_displacement']] = confounds.groupby(['subject', 'run'])[['dvars', 'framewise_displacement']].apply(lambda x: fillna(x))
confounds = confounds.set_index(['subject', 'run'])

confounds.to_pickle('./derivatives/all_subjects_confounds_with_gs.pkl')

/home/stevenm/.conda/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:18: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



## Extract non-STN timeseries
**Not used in paper, only for some control analyses**

In [2]:
!ls ./derivatives/ds-*/extracted_signal/sub-*/func/sub-*_task-randomdotmotion_run-*_space-*MNI*.csv

./derivatives/ds-01/extracted_signal/sub-01/func/sub-01_task-randomdotmotion_run-01_space-MNI152NLin2009cAsym_desc-preproc_desc-atlasROIs.csv
./derivatives/ds-01/extracted_signal/sub-01/func/sub-01_task-randomdotmotion_run-02_space-MNI152NLin2009cAsym_desc-preproc_desc-atlasROIs.csv
./derivatives/ds-01/extracted_signal/sub-01/func/sub-01_task-randomdotmotion_run-03_space-MNI152NLin2009cAsym_desc-preproc_desc-atlasROIs.csv
./derivatives/ds-01/extracted_signal/sub-02/func/sub-02_task-randomdotmotion_run-01_space-MNI152NLin2009cAsym_desc-preproc_desc-atlasROIs.csv
./derivatives/ds-01/extracted_signal/sub-02/func/sub-02_task-randomdotmotion_run-02_space-MNI152NLin2009cAsym_desc-preproc_desc-atlasROIs.csv
./derivatives/ds-01/extracted_signal/sub-02/func/sub-02_task-randomdotmotion_run-03_space-MNI152NLin2009cAsym_desc-preproc_desc-atlasROIs.csv
./derivatives/ds-01/extracted_signal/sub-03/func/sub-03_task-randomdotmotion_run-01_space-MNI152NLin2009cAsym_desc-preproc_desc-atlasROIs.csv
./deri

In [13]:
## load all extracted signals
all_csvs = sorted(glob.glob('./derivatives/ds-*/extracted_signal/sub-*/func/*space-MNI*.csv'))

reg = re.compile('.*ds-(?P<ds>\d+)/extracted_signal/sub-(?P<sub>\d+)/func/sub-.*_task-randomdotmotion_run-(?P<run>\d+)_space-MNI152NLin2009cAsym_desc-preproc_desc-.*')
reg.match(all_csvs[1]).groupdict()

{'ds': '01', 'sub': '01', 'run': '02'}

In [32]:
df = []

# loop over .csv-files, adding signal row-by-row
for fn in all_csvs:
    d = reg.match(fn).groupdict()
    dat = pd.read_csv(fn, index_col=0).apply(lambda x: to_psc(x), axis=0)
    dat = dat.reset_index()
    dat['sub'] = d['sub']
    dat['run'] = d['run']
    dat['ds'] = d['ds']
    dat['t'] = dat['volume'] * 3
    del dat['volume']
#    dat = dat.
    
    df.append(dat)

df = pd.concat(df, axis=0)
df['ds'] = df['ds'].astype(int)
df['run'] = df['run'].astype(int)
df['sub'] = df['sub'].astype(int)

# adjust ds2 subject idx to ensure non-overlapping subject idx
max_sub_ds1 = df.loc[df.ds==1,'sub'].max()
df.loc[df.ds==2,'sub'] = df.loc[df.ds==2,'sub']+max_sub_ds1
df = df.reset_index().rename(columns={'sub': 'subject', 't': 'time'})
df = df.set_index(['subject', 'run', 'time'])
del df['index']
del df['ds']

df.to_pickle('./derivatives/all_subjects_roi_timeseries.pkl')